# Ген. алгоритм "Распределение задач"

### Суть задачи: распределить задачи (1000) между программистами (10) так, чтобы максимальное время выполнения всех задач стремилось к минимуму

In [1263]:
import numpy as np

In [ ]:
def mutation(child, task_count, rng, p_mutat):
    chance = rng.random()
    if chance <=p_mutat:
        mutation_num = rng.integers(1, 4)
        #* Мутация обменом генов
        if mutation_num == 1:
            for _ in range(rng.integers(0, task_count)):
                fchromosome, schromosome = rng.integers(0, task_count), rng.integers(0, task_count)
                child[fchromosome], child[schromosome] = child[schromosome], child[fchromosome]
            return child
        #* Мутация обращением
        elif mutation_num == 2:
            fslice = rng.integers(0, task_count)
            sslice = rng.integers(fslice, task_count)
            child[fslice:sslice] = child[fslice:sslice][::-1]
            return child
        #* Мутация перетасовкой
        elif mutation_num == 3:
            fslice = rng.integers(0, task_count)
            sslice = rng.integers(fslice, task_count)
            np.random.shuffle(child[fslice:sslice])
            return child
    return child

In [ ]:
def crossover(fparent, sparent, task_count, rng, p_cross):
    chance = round(rng.random(), 5)
    if chance <= p_cross:
        crossover_num = rng.integers(1, 4)
        #* Одноточечый кроссовер (скрещивание с одним срезом)
        if crossover_num == 1:
            gen_slice = rng.integers(0, task_count)
            fchild = np.concatenate([fparent[:gen_slice], sparent[gen_slice:]])
            schild = np.concatenate([sparent[:gen_slice], fparent[gen_slice:]])
            return fchild, schild
        #* Двухточечный кроссовер (скрещивание с двумя срезами)
        elif crossover_num == 2:
            fslice = rng.integers(0, task_count)
            sslice = rng.integers(fslice, task_count)
            fchild = np.concatenate([fparent[:fslice], sparent[fslice:sslice], fparent[sslice:]])
            schild = np.concatenate([sparent[:fslice], fparent[fslice:sslice], sparent[sslice:]])
            return fchild, schild
        #* Равномерное скрещивание
        elif crossover_num == 3:
            return [[rng.choice([fparent[gen], sparent[gen]]) for gen in range(task_count)] for _ in range(2)]
    return fparent, sparent

Данный блок selection работает хуже, так как лучшие особи могут пропускаться, из-за чего алгоритм отбора в последующих генерациях будет выдавать хучшие результаты и программе придется искать новые индивиды

In [ ]:
# def selection(population, fitness_res, rng):
#     selection_num = rng.integers(1,4)
#     #* Стандартная выборка лучших особей в случайном срезе
#     if selection_num == 1:
#         return [ind for ind, _ in sorted(zip(population, fitness_res), key=lambda x: x[1])][:rng.choice([14, 22, 36, 42, 54])]
#     #* Случайная выборка особей в случайном диапазоне
#     elif selection_num == 2:
#         return [rng.choice(population, replace=False) for _ in range(rng.choice([14, 22, 36, 42, 54]))]
#     #* Выборка особей в перемешанном варианте в случайном диапазоне
#     elif selection_num == 3:
#         np.random.shuffle(population)
#         fslice, sslice = rng.choice([[14,36], [148, 192], [246, 304], [574, 620], [812, 888]])
#         return population[fslice:sslice]

In [ ]:
def selection(population, fitness_res, rng):
    return [ind for ind, _ in sorted(zip(population, fitness_res), key=lambda x: x[1])][:rng.choice([14, 22, 36, 42, 54])]

In [ ]:
def fitness(lvl_time, developer_count, developer_skills, individ):
    total_time = np.zeros(developer_count)
    
    for idx, ind in enumerate(individ):
        total_time[ind] += developer_skills[ind][int(lvl_time[idx][0])-1]*lvl_time[idx][1]
    return total_time.max()         #* Время окончания работы над проектом

In [ ]:
def genetic_algoritm(task_count, lvl_time, developer_count, developer_skills, population_size, generation_size, p_cross, p_mutat):
    rng = np.random.default_rng()
    population = rng.integers(0, developer_count, size=(population_size, task_count))
    
    for gen in range(generation_size):
        fitness_res = np.array([fitness(lvl_time, developer_count, developer_skills, individ) for individ in population])
        print(f'Поколение {gen}: Приспособленность - {fitness_res.min()}')

        parents = selection(population, fitness_res, rng)
        childrens = []
        for _ in range(0, population_size-len(parents), 2):
            fparent, sparent = rng.choice(parents, size=2, replace=False)
            fchild, schild = crossover(fparent, sparent, task_count, rng, p_cross)
            fmutated_child = mutation(fchild, task_count, rng, p_mutat)
            smutated_child = mutation(schild, task_count, rng, p_mutat)
            childrens.extend([fmutated_child, smutated_child])
        population = parents + childrens
    
    return population[fitness_res.argmin()]

In [1269]:
file = [rows.split() for rows in open('input.txt', 'r').readlines()]
task_count = int(*file[0])
lvl_time = np.array([np.array([int(lvl), float(time)]) for lvl, time in zip(file[1], file[2])])
developer_count = int(*file[3])
developer_skills = np.array([np.array(dev, dtype='float64') for dev in file[4:]])

In [1270]:
population_size = 1000          #* Количество хромосом (элементов) в популяции
generation_size = 300          #* Количество генераций (поколений / итераций)

p_cross = 0.775                 #* Шанс кроссовера (скрещивания)
p_mutat = 0.225                 #* Шанс мутации (изменения гена)

In [1271]:
solution = genetic_algoritm(task_count, lvl_time, developer_count, developer_skills, population_size, generation_size, p_cross, p_mutat)
output = open('output.txt', 'w').write(' '.join(map(str, solution)))

Поколение 0: Приспособленность - 681.3050000000003
Поколение 1: Приспособленность - 659.7400000000002
Поколение 2: Приспособленность - 638.0700000000003
Поколение 3: Приспособленность - 637.3000000000002
Поколение 4: Приспособленность - 633.2200000000001
Поколение 5: Приспособленность - 625.6000000000003
Поколение 6: Приспособленность - 625.5400000000001
Поколение 7: Приспособленность - 623.05
Поколение 8: Приспособленность - 618.0999999999999
Поколение 9: Приспособленность - 616.9999999999997
Поколение 10: Приспособленность - 615.7599999999996
Поколение 11: Приспособленность - 615.5550000000002
Поколение 12: Приспособленность - 615.5550000000002
Поколение 13: Приспособленность - 615.5550000000002
Поколение 14: Приспособленность - 615.5550000000002
Поколение 15: Приспособленность - 615.47
Поколение 16: Приспособленность - 615.3049999999997
Поколение 17: Приспособленность - 615.0399999999998
Поколение 18: Приспособленность - 614.0549999999998
Поколение 19: Приспособленность - 614.054999